In [ ]:
import pandas as pd
import numpy as np
# 导入数据
data=pd.read_excel('Call-Center-Dataset.xlsx')[['Call Id','Date']]
# 将时间设为索引
data.index=data['Date']
del data['Date']
# 按小时重采样
data=data.groupby(data.index.to_period('H')).count()
# 将索引变为时间序列
data.index=data.index.astype('datetime64[ns]')
# 填补缺失时间索引
date = pd.date_range('2015-01-01 09',periods=9,freq='H')
for day in pd.date_range('2015-01-02','2015-03-31',freq='D'):
    date=date.union(pd.date_range(str(day)[:10]+' 09',periods=9,freq='H'))
date
data=data.reindex(date)
# 填补缺失值
data=data.fillna(0)

In [ ]:
# 划分测试集和训练集
train=list(data.truncate(before='2015-01-01',after='2015-01-22')['Call Id'].values)
test=list(data.truncate(before='2015-01-22')['Call Id'].values)

In [ ]:


# 参数
s1=9
s2=7*9
alpha = 0.02
gamma = 0.02
delta = 0.01
omega = 0.2
phi = 0.8

# 平滑项 趋势项 两个季节项
St=[]
Tt=[]
Dt=[]
Wt=[]
y_hats=[]

# 选取初值
St.append(train[0])

T0=(train[-1]-train[0])/(len(train)-1)
Tt.append(T0)

for i in range(s1):
    si=train[i::s1]
    Dt.append(np.mean(si))

for i in range(s2):
    si=train[i::s2]
    Wt.append(np.mean(si))


# 拟合模型
for t in range(len(train)):
    st=alpha*(train[t]-Dt[-s1]-Wt[-s2])+(1-alpha)*(St[-1]+Tt[-1])
    St.append(st)

    tt=gamma*(St[-1]-St[-2])+(1-gamma)*Tt[-1]
    Tt.append(tt)

    dt=delta*(train[t]-St[-1]-Wt[-s2])+(1-delta)*Dt[-s1]
    Dt.append(dt)

    wt=omega*(train[t]-St[-1]-Dt[-s1-1])+(1-omega)*Wt[-s2]
    Wt.append(wt)

    y_hat=St[-1]+Tt[-1]+Dt[-s1]+Wt[-s2]+phi*(train[t]-(St[-2]+Tt[-2]+Dt[-s1-1]+Wt[-s2-1]))
    y_hats.append(y_hat)

# 计算MSE
MSE=np.mean((np.array(train)-np.array(y_hats))**2)
MSE

In [ ]:
# 预测
y_preds=[]
for t in range(len(test)):
    st=alpha*(y_hats[-1]-Dt[-s1]-Wt[-s2])+(1-alpha)*(St[-1]+Tt[-1])
    St.append(st)

    tt=gamma*(St[-1]-St[-2])+(1-gamma)*Tt[-1]
    Tt.append(tt)

    dt=delta*(y_hats[-1]-St[-1]-Wt[-s2])+(1-delta)*Dt[-s1]
    Dt.append(dt)

    wt=omega*(y_hats[-1]-St[-1]-Dt[-s1-1])+(1-omega)*Wt[-s2]
    Wt.append(wt) 

    if t==0:
        y_pred=St[-1]+Tt[-1]+Dt[-s1]+Wt[-s2]+phi*(train[-1]-(St[-2]+Tt[-2]+Dt[-s1-1]+Wt[-s2-1]))
    else:
        y_pred=St[-1]+Tt[-1]+Dt[-s1]+Wt[-s2]+phi*(y_preds[-1]-(St[-2]+Tt[-2]+Dt[-s1-1]+Wt[-s2-1]))
    y_preds.append(y_pred)

# 计算预测MSE
MSE_hat=np.mean((np.array(test)-np.array(y_preds))**2)
MSE_hat